In [1]:
import numpy as np

import selenium
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pickle
import psycopg2

from datetime import datetime
import time
import pprint
import gc

import os

In [2]:
os.chdir('Data')

In [3]:
params = {
    'host': 'localhost',
    'user': 'keith',
    'password': 'Password1@'
}

In [4]:
conn = psycopg2.connect(**params, dbname='jobs_links')

In [5]:
cur = conn.cursor()

In [6]:
datetime.strftime(datetime.today(), "%Y-%m-%d")

'2019-06-29'

In [18]:
# Can be run repeatedly and executed a single time
cur.execute("INSERT INTO job_listings_pages (date_pulled, city, link) VALUES ('2019-07-28', 'SF', 'test test test');")

In [19]:
conn.commit()

In [3]:
def get_soup(url):
    chrome_options = webdriver.chrome.options.Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path='../../../anaconda3/bin/chromedriver',
                              options=chrome_options)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup

In [4]:
def grab_job_links(base_url):
    urls = []
    
    soup = get_soup(base_url)
        
    for link in soup.find_all('div', {'class': 'title'}):
        try:
            partial_url = link.a.get('href')
            url = 'https://www.indeed.com' + partial_url
            urls.append(url)

        except:
            continue
        
    return urls

In [3]:
def grab_job_links(base_url):
    
    chrome_options = webdriver.chrome.options.Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path='../../../anaconda3/bin/chromedriver',
                              options=chrome_options)
    driver.get(base_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    
    urls = []
            
    for link in soup.find_all('div', {'class': 'title'}):
        try:
            partial_url = link.a.get('href')
            url = 'https://www.indeed.com' + partial_url
            urls.append(url)
            del url
            del partial_url

        except:
            continue
    
    del soup
        
    return urls

In [4]:
def get_jobs_pages(pickle_ins, pickle_outs):
    
    for p_in, p_out in zip(pickle_ins, pickle_outs):
        pickle_in = open(p_in, 'rb')
        job_listing_urls = pickle.load(pickle_in)
        pickle_in.close()
        
        urls = []
        for job_listing_url in job_listing_urls:
            urls.append(grab_job_links(job_listing_url))

            time.sleep(np.random.poisson(100)/50)

        pickle_out = open(p_out, 'wb')
        pickle.dump(urls, pickle_out)
        pickle_out.close()

In [5]:
pickle_ins = ['urls_to_query__LosAngeles',
              'urls_to_query__NewYork,+NY&_ga=',
              'urls_to_query__SanFranciscoBayArea%2C+CA',
              'urls_to_query__Seattle',
              'urls_to_query__Chicago']
pickle_outs = ['individual_listings__LosAngeles',
               'individual_listings__NewYork',
               'individual_listings__SanFranciscoBayArea',
               'individual_listings__Seattle',
               'individual_listings__chicago']

In [6]:
get_jobs_pages(pickle_ins, pickle_outs)

OSError: [Errno 12] Cannot allocate memory

## Looking at Links

In [17]:
pickle_in = open('individual_listings__chicago','rb')
Chicago_job_links = pickle.load(pickle_in)
pickle_in.close()

In [18]:
pickle_in = open('individual_listings__SanFranciscoBayArea','rb')
SF_job_links = pickle.load(pickle_in)
pickle_in.close()

In [19]:
pickle_in = open('individual_listings__NewYork','rb')
NY_job_links = pickle.load(pickle_in)
pickle_in.close()

In [20]:
pickle_in = open('individual_listings__LosAngeles','rb')
LA_job_links = pickle.load(pickle_in)
pickle_in.close()

In [21]:
pickle_in = open('individual_listings__Seattle','rb')
SA_job_links = pickle.load(pickle_in)
pickle_in.close()

In [26]:
pp = pprint.PrettyPrinter()

In [28]:
pp.pprint(Chicago_job_links[0][0])
pp.pprint(SF_job_links[0][0])
pp.pprint(NY_job_links[0][0])
pp.pprint(LA_job_links[0][0])
pp.pprint(SA_job_links[0][0])

'https://www.indeed.com/rc/clk?jk=e954b433367813e5&fccid=5acb3ef7d5f88321&vjs=3'
'https://www.indeed.com/company/Retrace-Labs/jobs/Data-Scientist-c6addb997485c790?fccid=3b8917bd1a013414&vjs=3'
'https://www.indeed.com/rc/clk?jk=cf5053055ba52a6a&fccid=dc461f3a0cfc5bbb&vjs=3'
'https://www.indeed.com/rc/clk?jk=7bd2ad2ec8e3456b&fccid=dd616958bd9ddc12&vjs=3'
'https://www.indeed.com/rc/clk?jk=ca79ed435ca66c1e&fccid=734cb5a01ee60f80&vjs=3'
